# Workflows and Agents

This guide reviews common patterns for agentic systems. In describing these systems, it can be useful to make a distinction between "workflows" and "agents". One way to think about this difference is nicely explained [here](https://www.anthropic.com/research/building-effective-agents) by Anthropic:

<blockqoute>
Workflows are systems where LLMs and tools are orchestrated through predefined code paths. Agents, on the other hand, are systems where LLMs dynamically direct their own processes and tool usage, maintaining control over how they accomplish tasks.
</blockqoute>

Here is a simple way to visualize these differences:
<img src="./images/agent_workflow.png">

When building agents and workflows, LangGraph offers a number of benefits including persistence, streaming, and support for debugging as well as deployment.

## Set up

You can use [any chat model](https://python.langchain.com/docs/integrations/chat/) that supports structured outputs and tool calling. Below, we show the process of installing the packages, setting API keys, and testing structured outputs / tool calling for Anthropic.

Initialize an LLM

In [1]:
import os
# import getpass

# from langchain_anthropic import ChatAnthropic

# def _set_env(var: str):
#     if not os.environ.get(var):
#         os.environ[var] = getpass.getpass(f"{var}: ")


# _set_env("ANTHROPIC_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

import keyring

OPENAI_API_KEY = keyring.get_password('openai', 'key_for_mac')
ANTHROPIC_API_KEY = keyring.get_password('anthropic', 'key_for_mac')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['ANTHROPIC_API_KEY'] = ANTHROPIC_API_KEY

llm = ChatOpenAI(model='gpt-4o-mini')
# llm = ChatAnthropic(model='claude-3-5-sonnet-latest')

## Building Blocks: The Augmented LLM

LLM have [augmentations](https://www.anthropic.com/research/building-effective-agents) that support building workflows and agents. These include [structured outputs](https://python.langchain.com/docs/concepts/structured_outputs/) and [tool calling](https://python.langchain.com/docs/concepts/tool_calling/), as shown in this image from the Anthropic [blog](https://www.anthropic.com/research/building-effective-agents):

<img src="./images/augmented_llm.png">

In [6]:
# Schema for structured output
from pydantic import BaseModel, Field

class SearchQuery(BaseModel):
    search_query: str = Field(None, description="Query that is optimized web search.")
    justification: str = Field(
        None, description="Why this query is relevant to the user's request."
    )

# Augment the LLM with schema for structured output
structured_llm = llm.with_structured_output(SearchQuery)

# Invoke the augmented LLM
output = structured_llm.invoke("How does Calcium CT score relate to high cholesterol?")
print(output)
print('-' * 80)

# Define a tool
def multiply(a: int, b: int) -> int:
    return a * b

# Augment the LLM with tools
llm_with_tools = llm.bind_tools([multiply])

# Invoke the LLM with input that triggers the tool call
msg = llm_with_tools.invoke("What is 2 times 3?")

# Get the tool call
msg.tool_calls

search_query='Calcium CT score and high cholesterol relationship' justification='This query is relevant as it seeks to explore the connection between the Calcium CT score, which indicates the presence of coronary artery disease, and high cholesterol levels, which are a significant risk factor for heart disease.'
--------------------------------------------------------------------------------


[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'call_z8IsJ1eZR8alcyfy7R3eq2HG',
  'type': 'tool_call'}]

## Prompt chaining